In [1]:
import pandas as pd
import pyodbc

In [2]:
sql_query = """
WITH work_order AS (
	SELECT *
	FROM [myANALYTICS_SP].[silver.fact.enterprise.asset].[vw_Fact_WorkOrder]
	WHERE
		BUKRS_CompanyCodeID = '5000'
),
all_floc AS (
    SELECT
        FLOC_TPLNR_FunctionalLocationInternalKey,
        FLOC_STRNO_FunctionalLocation,
        FLOC_PLTXT_FunctionalLocationDescr,
        --FLOC_TYPTX_FunctionalLocationCategoryDescr,
        --FLOC_EQART_TechnicalObjectType,
        FLOC_EARTX_TechnicalObjectTypeDescr,
        --FLOC_BUKRS_CompanyCode,
        --FLOC_STORT_Location,
        FLOC_STORT_LocationDescr
    FROM [myANALYTICS_SP].[silver.dimension.enterprise.asset].[vw_Dim_FunctionalLocation]
    WHERE FLOC_BUKRS_CompanyCode = '5000'
),
order_type AS (
	SELECT
		ORDTYP_AUART_OrderTypeKey,
		ORDTYP_TXT_OrderTypeDescr
	FROM [myANALYTICS_SP].[silver.dimension.enterprise.asset].[vw_Dim_OrderType]
),
wbs_element AS (
	SELECT
		[WBS_PSPNR_WBSElementInternalKey]
		,[WBS_PSPNR_WBSElement]
		,[WBS_PSPNR_EXT_WBSElement]
		,[WBS_POST1_WBSElementDescr]
		--,[WBS_PSPHI_ProjectInternal]
		,[WBS_POSKI_Project]
		,[WBS_POST1_ProjectDescr]
	FROM [myANALYTICS_SP].[silver.dimension.enterprise.finance].[vw_Dim_WBSElement]
),
cost_center AS (
	SELECT
		[CC_KOSTL_CostCenterInternalKey]
		,[CC_KOSTL_CostCenter]
		,[CC_LTEXT_CostCenterDescr]
		,[CC_MCTXT_CostCenterShortDescr]
		,[CC_DATBI_ValidToDate]
		,[CC_DATAB_ValidFromDAte]
		,[CC_BUKRS_CompanyCode]
		,[CC_KOSAR_CostCenterCategory]
		,[CC_VERAK_PersonResponsible]
		,[CC_VERAK_USER_UserResponsible]
		,[CC_PRCTR_ProfitCenterInternal]
		,[CC_KHINR_StandardHierarchyArea]
		,[CC_OBJNR_ObjectNumber]
		,[CC_FUNC_AREA_FunctionalArea]
		,[CC_BKZKP_LockActualPrimaryPostingFlag]
		,[CC_PKZKP_LockPlanPrimaryCostFlag]
		,[CC_ERSDA_CreatedOnDate]
		,[ZZPLANS_Position]
		,[CC_BKZKS_LockActualSecondaryCostFlag]
		,[CC_BKZER_LockActualRevenuePostingFlag]
		,[CC_BKZOB_LockCommitmentUpdateFlag]
		,[CC_PKZKS_LockPlanSecondaryCostFlag]
		,[CC_PKZER_LockPlanRevenuePostingFlag]
		,[CC_ABTEI_Department]
	FROM [myANALYTICS_SP].[silver.dimension.enterprise.finance].[vw_Dim_CostCenter]
)

SELECT
    work_order.AUFNR_Order,
    work_order.KTEXT_OrderDescr,
    work_order.ILART_MaintenanceActivityTypeID,
    work_order.GSTRP_BasicStartDateID,
    work_order.GLTRP_BasicFinishDateID,
    work_order.ActualCost,
    all_floc.FLOC_STRNO_FunctionalLocation,
    all_floc.FLOC_PLTXT_FunctionalLocationDescr,
    all_floc.FLOC_EARTX_TechnicalObjectTypeDescr,
    all_floc.FLOC_STORT_LocationDescr,
	work_order.AUART_OrderTypeID,
	order_type.ORDTYP_TXT_OrderTypeDescr,
	wbs_element.WBS_PSPNR_WBSElement,
	wbs_element.WBS_PSPNR_EXT_WBSElement,
	wbs_element.WBS_POST1_WBSElementDescr,
	wbs_element.WBS_POSKI_Project,
	wbs_element.WBS_POST1_ProjectDescr,
	cost_center.CC_KOSTL_CostCenter,
	cost_center.CC_LTEXT_CostCenterDescr
FROM work_order
    LEFT JOIN all_floc ON work_order.TPLNR_FunctionalLocationInternalID = all_floc.FLOC_TPLNR_FunctionalLocationInternalKey
	LEFT JOIN order_type ON work_order.AUART_OrderTypeID = order_type.ORDTYP_AUART_OrderTypeKey
	LEFT JOIN wbs_element ON work_order.PSPEL_WBSElementInternalID = wbs_element.WBS_PSPNR_WBSElementInternalKey
	LEFT JOIN cost_center ON work_order.KOSTV_ResponsibleCostCenterID = cost_center.CC_KOSTL_CostCenterInternalKey
ORDER BY work_order.GSTRP_BasicStartDateID
"""

# Define your server name
server_name = 'myanalytics.aurizon.com.au'

# Establish a connection using Windows Authentication
conn = pyodbc.connect('DRIVER={SQL Server};SERVER=' + server_name + ';Trusted_Connection=yes;')

# Execute the SQL query and load the result into a pandas DataFrame
df = pd.read_sql_query(sql_query, conn)

file_path = 'csv/work_orders_all.csv'
df.to_csv(file_path, index=True)

# Display the DataFrame
df

C:\Users\r893859\AppData\Local\Temp\ipykernel_17160\1426430670.py:57: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_all_work_orders = pd.read_sql_query(sql_query, conn)


,AUFNR_Order,KTEXT_OrderDescr,ILART_MaintenanceActivityTypeID,GSTRP_BasicStartDateID,GLTRP_BasicFinishDateID,ActualCost,FLOC_STRNO_FunctionalLocation,FLOC_PLTXT_FunctionalLocationDescr,FLOC_EARTX_TechnicalObjectTypeDescr,FLOC_STORT_LocationDescr
0,50411005,FY22 Inventory Management BW Gladstone,A07,20210701,20220630,8716.56,None,None,None,None
1,30687481,EXTRA LOW VOLTAGE INSPECT 6 MONTH,P06,20210701,20210701,331.87,PN-SP-FL-CFL101,CFL101 FORKLIFT PRAMAC LIFTER,Plant: Forklift,None
2,80104553,NW_IV826_OH_Headspan_Elec Design,E13,20210701,20210701,4527.00,None,None,None,None
3,58164905,Traction Power Systems Site - F Svc,NSV,20210701,20210701,396.05,ST001610-01,BO 60.5 Km PSC,Substation,Bolingbroke
4,80104554,NW_IV826_OH_FW-Edungalba_Elec Dgn,RCP,20210701,20210701,35975.00,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...
243497,600001393,Warren 46A C13,C17,20260701,20270630,0.00,TO000697,25.444KM 46A TURNOUT WARREN,Electrical Turnout,Warren
243498,600001392,Warren 45A C13,C17,20260701,20270630,0.00,TO000698,25.541KM 45A TURNOUT WARREN,Electrical Turnout,Warren
243499,30714563,XQ05RH RRV5605 HIRAIL BATTERY CHANGE 3YR,P06,20270301,20270301,0.00,PN-SP-HR-RR5605,RRV5605 ISUZU RRV TRUCK 7.5T XQ05RH,None,Blackwater Infrastructure Maintenance
243500,57006432,Ingsdon Dn P & C weld vee 101A,NRP,20300101,20300101,0.00,TO000362,11.251KM 101A TURNOUT INGSDON,Electrical Turnout,Ingsdon
